In [1]:
import json
import pandas as pd
import pickle

In [68]:
#j = json.load(open('/Users/ashton/school/cmsc828m/project/data/nvdcve-1.0-2017.json'))
del j

In [55]:
def cve_matches_name(cve, name):
    for vendor in cve['cve']['affects']['vendor']['vendor_data']:
        for product in vendor['product']['product_data']:
            if product['product_name'] == name:
                return True
    return False



25

In [60]:
def find_vulns(product_name): #version_str):
    results = []
    for year in range(10,19):
        j = json.load(open('/Users/ashton/school/cmsc828m/project/data/nvd_json/nvdcve-1.0-20' + str(year) + '.json'))
        for cve in j["CVE_Items"]:
            #if j["CVE_Items"][i]["cve"]["CVE_data_meta"]['ID'] in cve_ids:
            #    results.append(j["CVE_Items"][i])
            if cve_matches_name(cve, product_name):
                results.append(cve)
        del j
    print("unfiltered list: ", len(results))
    return results

def find_versions(all_vulns, version_str):
    results2 = []
    for r in all_vulns:
        versions = []
        for vd in r['cve']['affects']['vendor']['vendor_data']:
            for pd in vd['product']['product_data']:
                for version in pd['version']['version_data']:
                    #print version
                    versions.append(version['version_value'])
        if version_str in versions:
            results2.append({"is" : r['impact']['baseMetricV2']['impactScore'], 
                             "es" : r["impact"]['baseMetricV2']["exploitabilityScore"],
                             "bs" : r["impact"]['baseMetricV2']["cvssV2"]["baseScore"],
                             "cve_id" : r["cve"]["CVE_data_meta"]["ID"]})
    #results[0]['cve']['affects']['vendor']['vendor_data']#['version_data']
    print("final filtered: ", len(results2))
    return results2

django_cves = find_vulns("php")
len(django_cves)
#django_1_7_cves = find_versions(django_cves, "1.7")

('unfiltered list: ', 318)


318

In [89]:
def get_attacks(attacker_skill_level, attack_set):
    return [a for a in attack_set if a["es"] <= attacker_skill_level]

def get_attacks_rel(percentile, attack_set):
    return sorted(attack_set, key=lambda x: x['es'])[:int(len(attack_set) * percentile)]

#get_attacks(7, django_1_7_cves)

In [81]:
product_cves = {}
for product in ['mongodb', 'php', 'django', 'postgresql']:
    product_cves[product] = find_vulns(product)    

('unfiltered list: ', 12)
('unfiltered list: ', 318)
('unfiltered list: ', 57)
('unfiltered list: ', 55)


In [82]:
versions = {'django' : "1.7", 
            "mongodb" :  "2.4.3",
            "php" : "7.0.0" ,
            "postgresql" : "9.5"}

In [83]:
product_version_cves = {}
for k, v in versions.items():
    product_version_cves[k] = find_versions(product_cves[k], v)

#product_version_cves['php']

('final filtered: ', 4)
('final filtered: ', 85)
('final filtered: ', 16)
('final filtered: ', 19)


In [111]:
rewards = []
attacker_skill = [0.5, 0.75, 1]
services = []
for s in versions.keys():
    services.append(s)
    theta = []
    for a in attacker_skill:
        theta.append(get_attacks_rel(a, product_version_cves[s]))
    rewards.append(theta)
    

In [112]:
rewards[0]

[[{'bs': 2.1, 'cve_id': u'CVE-2016-6494', 'es': 3.9, 'is': 2.9},
  {'bs': 6.5, 'cve_id': u'CVE-2013-3969', 'es': 8.0, 'is': 6.4}],
 [{'bs': 2.1, 'cve_id': u'CVE-2016-6494', 'es': 3.9, 'is': 2.9},
  {'bs': 6.5, 'cve_id': u'CVE-2013-3969', 'es': 8.0, 'is': 6.4},
  {'bs': 6.5, 'cve_id': u'CVE-2013-4650', 'es': 8.0, 'is': 6.4}],
 [{'bs': 2.1, 'cve_id': u'CVE-2016-6494', 'es': 3.9, 'is': 2.9},
  {'bs': 6.5, 'cve_id': u'CVE-2013-3969', 'es': 8.0, 'is': 6.4},
  {'bs': 6.5, 'cve_id': u'CVE-2013-4650', 'es': 8.0, 'is': 6.4},
  {'bs': 4.3, 'cve_id': u'CVE-2013-2132', 'es': 8.6, 'is': 2.9}]]

In [108]:
pickle.dump(rewards, open('/Users/ashton/school/cmsc828m/project/data/attacker_actions/20180407-rewards.pickle', 'wb'))

In [117]:
rewards = []
attacker_skill = [6, 8, 10]
services = []
for s in versions.keys():
    services.append(s)
    theta = []
    for a in attacker_skill:
        theta.append(get_attacks(a, product_version_cves[s]))
    rewards.append(theta)
    

In [127]:
pickle.dump(rewards, open('/Users/ashton/school/cmsc828m/project/data/attacker_actions/20180411-rewards.pickle', 'wb'))

In [123]:
rewards[2][0]

[{'bs': 4.6, 'cve_id': u'CVE-2016-5424', 'es': 3.9, 'is': 6.4},
 {'bs': 7.2, 'cve_id': u'CVE-2017-12172', 'es': 3.9, 'is': 10.0}]

In [124]:
services

['mongodb', 'php', 'postgresql', 'django']

In [126]:
[(service, len(product_version_cves[service])) for service in services]

[('mongodb', 4), ('php', 85), ('postgresql', 16), ('django', 19)]